In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer
import nltk


mylist = list()
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy','Rank'),index=range(30000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
        
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv("Same_SIM_Master_List.csv")
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv("Same-Sim.csv",encoding ='latin1')
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv("Same-Sim.csv",encoding ='latin1')
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetSameSIMData():
    SameSimDf = pd.read_csv("Same-Sim.csv",encoding ='latin1')
    sNewCodeDesc = SameSimDf['SimCode'].tolist()
    return sNewCodeDesc;
   
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,RankNo,x):
    df = pd.read_csv("Same_SIM_Master_List.csv")    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    resultFrame.loc[x].Rank = RankNo
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
SameSimSet = pd.Series(GetSameSIMData())

    
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    test_set = pd.Series(i)
    #print(i)
    tokenizer = TfidfVectorizer(tokenizer=tokenize,stop_words = 'english') # initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
    score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
    #score
    pd.set_option("display.max_rows", 11)
    sortedscore = score.sort_values(by=['score'], ascending=False)
    sortedscore.head(11)
    ranknum = 0
    for x in range(0,10):
        ranknum = ranknum+1
        df = pd.read_csv("Same_SIM_Master_List.csv") 
        svalue = sortedscore.iloc[x,1]
        GetCPTValue = df.iloc[svalue,0]
        if (GetCPTValue == SameSimSet[NewCodeNum]):
            break 
    
    RecNum =0
    for x in range(0,10):
        rownum = rownum+1
        svalue = sortedscore.iloc[x,1]
        Accuracy = sortedscore.iloc[x,2]        
        if (RecNum ==0):
            result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,ranknum,rownum)
        else:
            result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,"",rownum)
        RecNum =RecNum +1
 
resultFrame

export_csv = resultFrame.to_csv ('export_dataframewithStemmingFull.csv', index = None, header=True)



In [15]:
textblob_tokenizer("fishes")

['fish']

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from nltk.corpus import stopwords
import numpy as np
import numpy.linalg as LA
import pandas as pd
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer
import nltk


mylist = list()
resultFrame = pd.DataFrame(columns=('NewCode','NewCodeDesc','SameSimCode', 'CPTDesc','Accuracy','Rank'),index=range(30000))
rownum = -1
NewCodeNum = -1

def create_tokenizer_score(new_series, train_series, tokenizer):
    """
    return the tf idf score of each possible pairs of documents
    Args:
        new_series (pd.Series): new data (To compare against train data)
        train_series (pd.Series): train data (To fit the tf-idf transformer)
    Returns:
        pd.DataFrame
    """
        
    train_tfidf = tokenizer.fit_transform(train_series)
    
    new_tfidf = tokenizer.transform(new_series)

    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_series.index)
    X['ix_new'] = new_series.index
    score = pd.melt(
        X,
        id_vars='ix_new',
        var_name='ix_train',
        value_name='score'
    )
    return score


def Get_MasterCodeDesc():
    df = pd.read_csv("Same_SIM_Master_List.csv")
    sMasterCodeDesc = df['CPTLongDesc'].tolist()
    return sMasterCodeDesc;

def GetNewCPT():
    NewCodeFrame = pd.read_csv("Same-Sim_SAMPLE.csv",encoding ='latin1')
    sNewCode = NewCodeFrame['HCPCS'].tolist()
    return sNewCode;

def GetNewCPTDesc():
    NewCodeFrame = pd.read_csv("Same-Sim_SAMPLE.csv",encoding ='latin1')
    sNewCodeDesc = NewCodeFrame['LongDesc'].tolist()
    return sNewCodeDesc;

def GetSameSIMData():
    SameSimDf = pd.read_csv("Same-Sim_SAMPLE.csv",encoding ='latin1')
    sNewCodeDesc = SameSimDf['SimCode'].tolist()
    return sNewCodeDesc;
   
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems

def GetMasterCPTCode(Val,NewCode,NewCodeDesc,Accuracy,RankNo,x):
    df = pd.read_csv("Same_SIM_Master_List.csv")    
    GetCPTValue = df.iloc[Val,0]
    GetCPTDesc = df.iloc[Val,1]    
    mylist.append(GetCPTValue)
    mylist.append(GetCPTDesc)
    resultFrame.loc[x].NewCode = NewCode
    resultFrame.loc[x].SameSimCode = GetCPTValue
    resultFrame.loc[x].CPTDesc = GetCPTDesc
    resultFrame.loc[x].NewCodeDesc = NewCodeDesc
    resultFrame.loc[x].Accuracy = Accuracy
    resultFrame.loc[x].Rank = RankNo
    return mylist;

#train_CPT_Set = pd.Series(GetMasterCPT())
train_set = pd.Series(Get_MasterCodeDesc())
NewCodeDescSet = pd.Series(GetNewCPTDesc())
NewCodeSet = pd.Series(GetNewCPT())
SameSimSet = pd.Series(GetSameSIMData())

    
for i in NewCodeDescSet:
    NewCodeNum = NewCodeNum+1
    test_set = pd.Series(i)
    #print(i)
    tokenizer = TfidfVectorizer(stop_words = 'english') # initiate here your own tokenizer (TfidfVectorizer, CountVectorizer, with stopwords...)
    score = create_tokenizer_score(train_series=train_set, new_series=test_set, tokenizer=tokenizer)
    #score
    pd.set_option("display.max_rows", 11)
    sortedscore = score.sort_values(by=['score'], ascending=False)
    sortedscore.head(11)
    ranknum = 0
    for x in range(0,10):
        ranknum = ranknum+1
        df = pd.read_csv("Same_SIM_Master_List.csv") 
        svalue = sortedscore.iloc[x,1]
        GetCPTValue = df.iloc[svalue,0]
        if (GetCPTValue == SameSimSet[NewCodeNum]):
            break 
    
    RecNum =0
    for x in range(0,10):
        rownum = rownum+1
        svalue = sortedscore.iloc[x,1]
        Accuracy = sortedscore.iloc[x,2]        
        if (RecNum ==0):
            result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,ranknum,rownum)
        else:
            result = GetMasterCPTCode(svalue,NewCodeSet[NewCodeNum],i,Accuracy*100,"",rownum)
        RecNum =RecNum +1
 
resultFrame

export_csv = resultFrame.to_csv ('export_dataframeWithoutStemming.csv', index = None, header=True)

